# Evaluate Kahoot performance during a course

## Evaluate points

In [1]:
import os
import pandas
import numpy as np

In [2]:
# Initialize same random state every time
random_state = np.random.RandomState(20190925)

### Read data from Excell files

In [3]:
directory_name = "A:\\1docs\\aktualis\\2019-2020-1 Oktatas\\Korszeru_nuklearis_energiatermeles\\kahoot"
sheet_name = "Final Scores"
list_of_all_files = os.listdir(directory_name)
list_of_all_files

['kahoot-1-1.xlsx',
 'kahoot-2-3.xlsx',
 'kahoot-2-2.xlsx',
 'kahoot-2-1.xlsx',
 'kahoot-3-2.xlsx',
 'kahoot-3-1.xlsx',
 'kahoot-4-1.xlsx',
 'kahoot-4-2.xlsx',
 'kahoot_results.xlsx',
 'kahoot-5-1.xlsx']

In [4]:
list_of_files = [elem for elem in list_of_all_files if elem.find("kahoot-")==0]
list_of_files

['kahoot-1-1.xlsx',
 'kahoot-2-3.xlsx',
 'kahoot-2-2.xlsx',
 'kahoot-2-1.xlsx',
 'kahoot-3-2.xlsx',
 'kahoot-3-1.xlsx',
 'kahoot-4-1.xlsx',
 'kahoot-4-2.xlsx',
 'kahoot-5-1.xlsx']

### Players corrections

In [5]:
def convert_players(old_player):
    if old_player == "":
        return ""
    old_player = str(old_player).upper()
    return old_player

conv_players=np.vectorize(convert_players)

### Read data from Excell files and perform transformations

In [6]:
scores = pandas.DataFrame()
for file_name in list_of_files:
    scores_single = pandas.read_excel(directory_name+"\\"+file_name, sheetname=sheet_name, header=1, 
                                      skiprows=[1], skipfooter=1,
                                      converters={"Players":convert_players})
    # Give points for perfect tests
    scores_single["Points"] = scores_single["Incorrect Answers"]==0 
    # Give points to two further tests randomly selected with Kahoot points acting as weights 
    weights = scores_single[scores_single["Points"]==False]["Total Score (points)"].values
    weights = weights / sum(weights)
    selected_ind = random_state.choice(scores_single[scores_single["Points"]==False].index,2,p=weights)
    scores_single.iloc[selected_ind,scores_single.columns.get_loc("Points")] = True
    # Append to summary table
    scores = scores.append(scores_single)

In [7]:
scores[scores["Points"]]

,Rank,Players,Total Score (points),Correct Answers,Incorrect Answers,Points
0,1,QBT6IE,7358,9,0,True
9,10,P3RTXR,5358,8,1,True
29,30,F1VZ66,3523,6,3,True
0,1,EE6J2I,4260,4,0,True
1,2,JQ8QL2,4122,4,0,True
2,3,Y7IFMA,4093,4,0,True
3,4,A5AILB,4034,4,0,True
4,5,Q55SBJ,3824,4,0,True
5,6,HZWCET,3813,4,0,True
6,7,P3RTXR,3796,4,0,True


### Aggregate results of individual Players

In [8]:
scores_aggregate = scores.groupby('Players',as_index=False).agg({'Rank': ['mean', 'count'],
                                                                'Total Score (points)': 'sum',
                                                                'Correct Answers': 'sum',
                                                                'Incorrect Answers': 'sum',
                                                                'Points': 'sum'})

In [9]:
scores.columns

Index(['Rank', 'Players', 'Total Score (points)', 'Correct Answers',
       'Incorrect Answers', 'Points'],
      dtype='object')

In [10]:
scores_aggregate.columns=[ 'Players', 'Rank', 'Count', 'Total Score (points)', 'Correct Answers', 
                          'Incorrect Answers', 'Points']


In [11]:
scores_aggregate

,Players,Rank,Count,Total Score (points),Correct Answers,Incorrect Answers,Points
0,A5AILB,17.222222,9,50267,53,16,2.0
1,BFJBWY,14.833333,6,31936,33,8,2.0
2,BTDI1A,14.857143,7,45699,45,10,2.0
3,BXAW5O,9.285714,7,43326,45,6,4.0
4,BXKWRC,13.500000,8,45956,48,17,1.0
5,CPOXGJ,12.375000,8,55574,53,7,4.0
6,DJTR9H,19.833333,6,25928,32,14,1.0
7,DWQC5C,16.375000,8,35347,40,15,1.0
8,EAXG4Z,20.833333,6,30505,34,17,1.0
9,EE6J2I,8.875000,8,45169,46,9,4.0


### Print result

In [12]:
writer = pandas.ExcelWriter(directory_name+"\\"+'kahoot_results.xlsx')
scores_aggregate.to_excel(writer,'Results')
writer.save()

## Evaluate question statistics